In [1]:
from DiSuQ.Torch import models
from scipy.optimize import minimize
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,logspace,std,sqrt,log10,tile
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox,plotOptimization
from DiSuQ.Torch.optimization import domainParameters
from time import perf_counter,sleep
from DiSuQ.Torch.components import indE,capE
import pandas

In [2]:
L0 = indE(.3e-9); L_ = indE(.3e-6); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(.5e-12); C_ = capE(1e-9) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(.5e-12); CJ_ = capE(1e-9) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 500. ; J_ = .1 ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 0.5448717098327724 545.4165815426054
Capacitance Bound(GHz): 1.937022930770759e-05 0.03875982884472289
Shunt Bound(GHz): 1.937022930770759e-05 0.03875982884472289
Junction Bound(GHz): 0.1 500.1


In [18]:
Lx = array([.8,24,128])
Cx = array([5e-4,5e-2,.01])
Jx = array([.05,10.,100.])
CJx = array([5e-4,5e-2,.01])

In [19]:
def representationCapacity(El,Ej,Ec,EcJ):
    basis = {'Chi':7,'Theta':64,'Phi':1024}
    # theta capacity is sufficient for the given parameter range
    basis = {'Chi':7,'Theta':12,'Phi':1024}
    flux0 = 24*pi*2
    return flux0,basis

In [20]:
basis = {'Chi':7,'Theta':12,'Phi':30}
rep = 'R'; flux_point = ['Lx','Ly']
subspace = ['Lx','Cx','Jx','CJx']
circuit = models.zeroPi(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,
                        symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0),
                        ridge=True,flux0=24*pi)
initials = domainParameters([Lx,Cx,Jx,CJx],circuit,subspace)
len(initials)

81

In [6]:
Bounds = []; components = circuit.circuitComposition()
for iD in subspace:
    bound = components[iD].bounds()
    Bounds.append(bound)

In [7]:
flux_range = linspace(0,.05,3,endpoint=True)
D0 = 2/3 ; delta0 = 1/3

In [8]:
def metric(parameters):
    El,Ec,Ej,EcJ = parameters
    flux0,basis = representationCapacity(El,Ej,Ec,EcJ)
    zero_pi = models.SCzeroPi(basis,flux0,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ)
    spectrum = zero_pi.get_spectrum_vs_paramvals('flux',flux_range,
                                                 evals_count=3)
    G,I,II = spectrum.energy_table.T
    D = log10((II[0]-G[0])/(I[0]-G[0]))
    delta = log10((I[0]-G[0])/std(I-G))
    return D,delta,(G,I,II)

def objective(parameters):
    D,delta,(G,I,II) = metric(parameters)
    return -D*D0 - delta*delta0

In [9]:
class CallbackFunctor:
    def __init__(self,D0,delta0,subspace):
        self.dLogs = []
        self.dCircuit = []
        self.subspace = subspace
        self.D0 = D0
        self.delta0 = delta0
        self.start = perf_counter()
        
    def __call__(self,parameters):
        self.dCircuit.append(dict(zip(subspace,parameters)))
        D,delta,(G,I,II) = metric(parameters)
        metrics = dict()
        loss = -D*self.D0 - delta*self.delta0
        metrics['loss'] = loss
        metrics['E10'] = I[0]-G[0]
        metrics['E20'] = II[0]-G[0]
        metrics['D'] = D; metrics['delta'] = delta
        metrics['time'] = perf_counter() - self.start
        self.dLogs.append(metrics)
        
    def logs(self):
        dLog = pandas.DataFrame(self.dLogs)
        if len(dLog)>0:
            dLog['time'] = dLog['time'].diff()
        else:
            print('Failure initial')
        dCircuit = pandas.DataFrame(self.dCircuit)
        return dLog,dCircuit

In [10]:
method='L-BFGS-B'; options=dict(ftol=0,maxiter=15)
Search = []
for index,init in enumerate(initials):
    print(index,init)
    logger = CallbackFunctor(D0,delta0,subspace)
    x0 = init['Lx'],init['Cx'],init['Jx'],init['CJx']
    results = minimize(objective,x0,method=method,options=options,
                       jac='3-point',bounds=Bounds,callback=logger)
    print(results)
    Search.append(logger.logs())

0 {'Jx': 0.05, 'Jy': 9.999998092651367, 'CJx': 0.0005, 'CJy': 0.038759831339120865, 'Lx': 0.8, 'Ly': 9.99999713897705, 'Cx': 0.0005, 'Cy': 0.038759831339120865}


Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

...

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

Spectral data:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
import pickle
with open('SCZeroSub.pickle', 'wb') as handle:
    pickle.dump(Search, handle, protocol=pickle.HIGHEST_PROTOCOL)